In [1]:
import itertools

import IPython.display
from matplotlib.axes import Axes
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
# df_orig = pd.read_csv('../logs/rundirs/sorted-20241116_215630.csv')
df_orig = pd.read_csv('../logs/rundirs/sorted-20241118_183738.csv')
df_orig

,Date,Scenario ID,Vehicle ID,Vehicle type,Cycle distance (m),No. of completed missions,Total distance traveled (m),No. of stops,No. of forcing events,No. of critical sections,No. of near-misses,No. of collisions,Is deadlocked,Total waiting time (s),Maximum waiting time (s),Total time (s),Maximum acceleration (m/s^2),Maximum speed (m/s),Average speed (m/s)
0,20241118_174623,map-generator/generated-maps/2024-11-16_17:00:...,0,HumanDrivenVehicle,77.56,6,470.4,13,0,108,2,0,0,192.4,93.6,442.5,0.3,10.0,1.1
1,20241118_174924,map-generator/generated-maps/2024-11-16_17:00:...,0,HumanDrivenVehicle,77.56,0,8.0,1,0,8,0,0,0,7.8,6.7,17.0,0.3,10.0,0.5
2,20241118_175105,map-generator/generated-maps/2024-11-16_17:00:...,0,HumanDrivenVehicle,77.56,8,647.4,19,0,149,6,0,1,263.1,93.6,599.9,0.3,10.0,1.1
3,20241118_175336,map-generator/generated-maps/2024-11-16_17:00:...,0,HumanDrivenVehicle,77.56,19,1517.9,20,14,179,2,6,0,15.9,1.5,599.9,0.3,10.0,2.5
4,20241118_175618,map-generator/generated-maps/2024-11-16_17:00:...,0,HumanDrivenVehicle,77.56,19,1466.8,20,17,183,3,6,0,22.1,7.9,599.9,0.3,10.0,2.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,20241118_182108,map-generator/generated-maps/2024-11-16_17:00:...,4,AutonomousVehicle,49.98,17,1743.2,18,0,114,1,0,0,14.5,1.9,599.9,0.3,10.0,2.9
91,20241118_182329,map-generator/generated-maps/2024-11-16_17:00:...,4,AutonomousVehicle,49.98,17,1762.6,18,0,113,1,0,0,14.5,1.9,599.9,0.3,10.0,2.9
92,20241118_182550,map-generator/generated-maps/2024-11-16_17:00:...,4,AutonomousVehicle,49.98,17,1728.2,18,0,116,2,0,0,14.5,1.9,599.9,0.3,10.0,2.9
93,20241118_182811,map-generator/generated-maps/2024-11-16_17:00:...,4,AutonomousVehicle,60.34,16,1945.3,16,0,72,0,0,0,11.9,0.7,599.9,0.3,10.0,3.2


In [3]:
df_id = df_orig['Scenario ID'].str.split(r'[;,] ', expand=True)
df_id.columns = ['filename', 'Coordination strategy', 'string_seed', 'string_probabilityForcingForHuman', 'heuristic']
df_id = pd.concat([
    df_id, 
    df_id['filename'].str.extract(r'/scenario(?P<i_map>\d+)-(?P<i_locations>\d+).json$', expand=True).astype(int),
    df_id['string_probabilityForcingForHuman'].str.extract(r'^probabilityForcingForHuman (?P<probabilityForcingForHuman>[\d.]+)$', expand=True).astype(float),
], axis=1).rename(columns={'i_locations': 'Positions variant'})
df_id

,filename,Coordination strategy,string_seed,string_probabilityForcingForHuman,heuristic,i_map,Positions variant,probabilityForcingForHuman
0,map-generator/generated-maps/2024-11-16_17:00:...,baseline,seed 1,probabilityForcingForHuman 0,AUTOMATED_FIRST,1,1,0.0
1,map-generator/generated-maps/2024-11-16_17:00:...,baseline,seed 1,probabilityForcingForHuman 0,AUTOMATED_FIRST,1,1,0.0
2,map-generator/generated-maps/2024-11-16_17:00:...,baseline,seed 1,probabilityForcingForHuman 0,AUTOMATED_FIRST,1,1,0.0
3,map-generator/generated-maps/2024-11-16_17:00:...,change of priorities,seed 1,probabilityForcingForHuman 1,AUTOMATED_FIRST,1,1,1.0
4,map-generator/generated-maps/2024-11-16_17:00:...,stops,seed 1,probabilityForcingForHuman 1,AUTOMATED_FIRST,1,1,1.0
...,...,...,...,...,...,...,...,...
90,map-generator/generated-maps/2024-11-16_17:00:...,baseline,seed 1,probabilityForcingForHuman 0,AUTOMATED_FIRST,1,5,0.0
91,map-generator/generated-maps/2024-11-16_17:00:...,change of priorities,seed 1,probabilityForcingForHuman 1,AUTOMATED_FIRST,1,5,1.0
92,map-generator/generated-maps/2024-11-16_17:00:...,stops,seed 1,probabilityForcingForHuman 1,AUTOMATED_FIRST,1,5,1.0
93,map-generator/generated-maps/2024-11-16_17:00:...,baseline,seed 1,probabilityForcingForHuman 0,AUTOMATED_FIRST,1,6,0.0


In [4]:
index_id = df_orig.columns.get_loc('Scenario ID')
df = pd.concat([df_id[['Coordination strategy', 'probabilityForcingForHuman', 'Positions variant']], df_orig.iloc[:, index_id + 1:]], axis=1)
df

,Coordination strategy,probabilityForcingForHuman,Positions variant,Vehicle ID,Vehicle type,Cycle distance (m),No. of completed missions,Total distance traveled (m),No. of stops,No. of forcing events,No. of critical sections,No. of near-misses,No. of collisions,Is deadlocked,Total waiting time (s),Maximum waiting time (s),Total time (s),Maximum acceleration (m/s^2),Maximum speed (m/s),Average speed (m/s)
0,baseline,0.0,1,0,HumanDrivenVehicle,77.56,6,470.4,13,0,108,2,0,0,192.4,93.6,442.5,0.3,10.0,1.1
1,baseline,0.0,1,0,HumanDrivenVehicle,77.56,0,8.0,1,0,8,0,0,0,7.8,6.7,17.0,0.3,10.0,0.5
2,baseline,0.0,1,0,HumanDrivenVehicle,77.56,8,647.4,19,0,149,6,0,1,263.1,93.6,599.9,0.3,10.0,1.1
3,change of priorities,1.0,1,0,HumanDrivenVehicle,77.56,19,1517.9,20,14,179,2,6,0,15.9,1.5,599.9,0.3,10.0,2.5
4,stops,1.0,1,0,HumanDrivenVehicle,77.56,19,1466.8,20,17,183,3,6,0,22.1,7.9,599.9,0.3,10.0,2.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,baseline,0.0,5,4,AutonomousVehicle,49.98,17,1743.2,18,0,114,1,0,0,14.5,1.9,599.9,0.3,10.0,2.9
91,change of priorities,1.0,5,4,AutonomousVehicle,49.98,17,1762.6,18,0,113,1,0,0,14.5,1.9,599.9,0.3,10.0,2.9
92,stops,1.0,5,4,AutonomousVehicle,49.98,17,1728.2,18,0,116,2,0,0,14.5,1.9,599.9,0.3,10.0,2.9
93,baseline,0.0,6,4,AutonomousVehicle,60.34,16,1945.3,16,0,72,0,0,0,11.9,0.7,599.9,0.3,10.0,3.2


In [5]:
series_deadlocks = df[df['Positions variant'] != 6].groupby(['Positions variant'])['Is deadlocked'].sum()
indices_locations_deadlocked = series_deadlocks[series_deadlocks != 0].index
indices_locations_nondeadlocked = series_deadlocks[series_deadlocks == 0].index
indices_locations_nondeadlocked

Index([2, 3, 4, 5], dtype='int64', name='Positions variant')

In [6]:
def save_and_show(filename):  # to avoid inlining large image data into the notebook file
    plt.savefig(filename)
    plt.close()
    IPython.display.display(IPython.display.HTML(f'<img src="{filename}" alt="{filename}" />'))

In [7]:
def plot_aut(df):
    # Extract levels
    positions = df.index.get_level_values('Positions variant').unique()
    strategies = df.index.get_level_values('Coordination strategy').unique()
    
    # Plot configuration
    fig, axes = plt.subplots(1, len(positions), figsize=(20, 6), sharey=True, squeeze=False)
    axes: list[list[Axes]]
    bar_width = 0.4
    colors = ['tab:blue', 'red', 'yellow']
    
    # Get the global min and max values for consistent y-axis scaling
    y1_min, y1_max = df['Total distance traveled (m)'].min(), df['Total distance traveled (m)'].max()
    y2_min, y2_max = min(df['No. of collisions'].min(), df['No. of near-misses'].min()), max(df['No. of collisions'].max(), df['No. of near-misses'].max())
    
    # Iterate through each Positions variant
    for i, pos in enumerate(positions):
        ax = axes[0][i]
        df_pos = df.loc[pos]
        
        # Bar positions for each Coordination strategy
        x_positions = np.arange(len(strategies))
        
        # Plot bars for 'Total distance traveled (m)'
        bars = ax.bar(x_positions, df_pos['Total distance traveled (m)'], width=bar_width, label='Distance Traveled', color=colors[0])
        ax.set_xlabel('Coordination Strategy')
        ax.set_ylabel('Total Distance Traveled (m)', color=colors[0])
        ax.tick_params(axis='y', labelcolor=colors[0])
        ax.set_ylim(0, y1_max * 1.1)
        
        # Create a secondary axis for the points
        ax_right = ax.twinx()
        
        # Plot points for 'No. of collisions' and 'No. of near-misses'
        collisions, = ax_right.plot(x_positions, df_pos['No. of collisions'], marker='o', linestyle='', label='No. of Collisions', color=colors[1])
        near_misses, = ax_right.plot(x_positions, df_pos['No. of near-misses'], marker='o', linestyle='', label='No. of Near Misses', color=colors[2])
        ax_right.set_ylabel('Collisions & Near Misses', color='black')
        ax_right.tick_params(axis='y', labelcolor='black')
        ax_right.set_ylim(0, y2_max * 1.1)
        
        # Add labels, grid, and title for each section
        ax.set_xticks(x_positions)
        ax.set_xticklabels(strategies, rotation=45, ha='right')
        ax.set_title(f'Positions variant {pos}')
        ax.grid(axis='y')
        
        if i == 0:
            handles = [bars, collisions, near_misses]
            labels = ['Distance Traveled', 'No. of Collisions', 'No. of Near Misses']
        
    fig.suptitle('Automated vehicles (summarised for AV1-AV4)', fontsize=16)
    fig.legend(handles=handles, labels=labels, loc='upper center', ncol=3, bbox_to_anchor=(0.5, 1.1))
    plt.tight_layout()    
    save_and_show('aut.png')

    
df_aut = df
df_aut = df_aut[df_aut['Positions variant'].isin(indices_locations_nondeadlocked)]
df_aut = df_aut[df_aut['Vehicle type'] == 'AutonomousVehicle']
df_aut = df_aut.groupby(['Positions variant', 'Coordination strategy']).sum()
df_aut = df_aut[['Total distance traveled (m)', 'No. of collisions', 'No. of near-misses']]
plot_aut(df_aut)
df_aut

Total distance traveled (m)  \
Positions variant Coordination strategy                                
2                 baseline                                    5726.0   
                  change of priorities                        5573.9   
                  stops                                       5130.6   
3                 baseline                                    5054.1   
                  change of priorities                        5118.8   
                  stops                                       4738.0   
4                 baseline                                    6166.4   
                  change of priorities                        6168.6   
                  stops                                       6024.8   
5                 baseline                                    5437.4   
                  change of priorities                        5494.1   
                  stops                                       5177.1   

                                         No. of collisions  No. of near-misses  
Positions variant Coordination strategy                                         
2                 baseline                               0                   4  
                  change of priorities                   5                   3  
                  stops                                  7                   2  
3                 baseline                               4                   4  
                  change of priorities                   6                   1  
                  stops                                  2                   0  
4                 baseline                               1                   2  
                  change of priorities                   2                   3  
                  stops                                  0                   4  
5                 baseline                               2                   2  
                  change of priorities                   2                   4  
                  stops                                  3                   7

In [8]:
def plot_hum(df):
    # Extract levels
    positions = df.index.get_level_values('Positions variant').unique()
    strategies = df.index.get_level_values('Coordination strategy').unique()
    
    # Plot configuration
    fig, axes = plt.subplots(1, len(positions), figsize=(13, 4), sharey=True, squeeze=False)
    axes: list[list[Axes]]
    bar_width = 0.4
    colors = ['tab:blue', 'orange']
    
    # Get the global min and max values for consistent y-axis scaling
    y1_min, y1_max = df['Total distance traveled (m)'].min(), df['Total distance traveled (m)'].max()
    y2_min, y2_max = df['No. of forcing events'].min(), df['No. of forcing events'].max()
    
    # Iterate through each Positions variant
    for i, pos in enumerate(positions):
        ax = axes[0][i]
        df_pos = df.loc[pos]
        
        # Bar positions for each Coordination strategy
        x_positions = np.arange(len(strategies))
        
        # Plot bars for 'Total distance traveled (m)'
        bars = ax.bar(x_positions, df_pos['Total distance traveled (m)'], width=bar_width, label='Distance Traveled', color=colors[0])
        ax.set_xlabel('Coordination Strategy')
        ax.set_ylabel('Total Distance Traveled (m)', color=colors[0])
        ax.tick_params(axis='y', labelcolor=colors[0])
        ax.set_ylim(0, y1_max * 1.1)
        
        # Create a secondary axis for the points
        ax_right = ax.twinx()
        
        # Plot points for 'No. of collisions' and 'No. of near-misses'
        forcing_events, = ax_right.plot(x_positions, df_pos['No. of forcing events'], marker='o', linestyle='', label='No. of forcing events', color=colors[1])
        ax_right.set_ylabel('forcing events', color='black')
        ax_right.tick_params(axis='y', labelcolor='black')
        ax_right.set_ylim(0, y2_max * 1.1)
        
        # Add labels, grid, and title for each section
        ax.set_xticks(x_positions)
        ax.set_xticklabels(strategies, rotation=45, ha='right')
        ax.set_title(f'Positions variant {pos}')
        ax.grid(axis='y')
        
        if i == 0:
            handles = [bars, forcing_events]
            labels = ['Distance Traveled', 'No. of forcing events']
        
    fig.suptitle('Human-driven vehicle', fontsize=16)
    fig.legend(handles=handles, labels=labels, loc='upper center', ncol=3, bbox_to_anchor=(0.5, 1.1))
    plt.tight_layout()
    save_and_show('hum.png')

    
df_hum = df
df_hum = df_hum[df_hum['Positions variant'].isin(indices_locations_nondeadlocked)]
df_hum = df_hum[df_hum['Vehicle type'] == 'HumanDrivenVehicle']
df_hum = df_hum.groupby(['Positions variant', 'Coordination strategy']).sum()
df_hum = df_hum[['Total distance traveled (m)', 'No. of forcing events']]
plot_hum(df_hum)
df_hum

Total distance traveled (m)  \
Positions variant Coordination strategy                                
2                 baseline                                     932.6   
                  change of priorities                        1465.2   
                  stops                                       1495.9   
3                 baseline                                    1180.2   
                  change of priorities                        1541.9   
                  stops                                       1544.7   
4                 baseline                                    1094.1   
                  change of priorities                        1538.6   
                  stops                                       1539.8   
5                 baseline                                    1314.1   
                  change of priorities                        1403.7   
                  stops                                       1468.5   

                                         No. of forcing events  
Positions variant Coordination strategy                         
2                 baseline                                   0  
                  change of priorities                      21  
                  stops                                     20  
3                 baseline                                   0  
                  change of priorities                       6  
                  stops                                      6  
4                 baseline                                   0  
                  change of priorities                       8  
                  stops                                      9  
5                 baseline                                   0  
                  change of priorities                      15  
                  stops                                     16

In [9]:
def show_maps(title, indices, nrows, ncols): 
    fig, axes_matrix = plt.subplots(nrows, ncols, figsize=(16, 9), squeeze=False)
    axes_matrix: list[list[Axes]]
    axes = list(itertools.chain.from_iterable(axes_matrix))
    assert len(axes) >= len(indices)
    
    for ax in axes:
        ax.axis('off')
        
    for ax, i_pos in zip(axes, indices):
        image = plt.imread(f'../screenshotting/screenshots/cropped/{i_pos}.png')
        ax.imshow(image)
        ax.title.set_text(f'Positions variant {i_pos}')
    
    fig.suptitle(title, fontsize=16)

    plt.subplots_adjust(wspace=0.1, hspace=0.05)
    save_and_show(title + '.png')
    
show_maps('Non-deadlocked', indices_locations_nondeadlocked, 2, 3)
show_maps('Deadlocked', indices_locations_deadlocked, 2, 3)